<a href="https://colab.research.google.com/github/icecore013/image-segmentation-keras/blob/master/1a_data_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective of the notebook

The aim of this code is to produce a training database containing the Sentinel 2 images and associated masks.

# Imports and configuration

In [ ]:
# Installing and importing the desired packages

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from google.colab import drive

import os
from os import path as op
import glob
import pprint
pp = pprint.PrettyPrinter(indent=4)
from tqdm import tqdm

import ee
import folium
import gdal
import ogr

!pip install rasterio
import rasterio as rio
from rasterio.windows import Window

!pip install geopandas
import geopandas as gpd

from pyproj import Proj, transform

In [ ]:
# Loading of our drive
drive.mount('/content/gdrive', force_remount=True)
root_path = 'gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=KVxTI021KhUJDvSfRQ8dIE6cmqY63G9WpxZmvNYzFP4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiNwwQvDju44pAjmVAy6QGDS9Mhj0NALkj2cp_qGAeBvM9cbYIwuhw

Successfully saved authorization token.


In [ ]:
# Setting the config variables

NUMBER_OF_FIRES = 500

DATABASE_PATH = "/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/"

DATABASE_NAME = "CAL_database"

SHAPEFILE_FOLDER = "raw_shapefiles_32610"
SENTINEL_FOLDER = "sentinel_tif_files"
LABEL_FOLDER = "label_tif_files"
INDIVIDUAL_SHAPEFILE_FOLDER = 'individual_shapefiles'

GDF_MAPPING_DICT = {
    'firenum': 'fire_id',
    'ALARM_DATE': 'fire_alert_date',
    'CONT_DATE': 'fire_cont_date',
    'SURFHA': 'fire_area',
    'geometry': 'fire_geometry'
}

INDIVIDUAL_SHAPEFILE_CRS_CONFIG = 'epsg:32610'

# Complete shapefile database collection

In [ ]:
# Global geo dataframe
shapefile_list = sorted(os.listdir(op.join(DATABASE_PATH, DATABASE_NAME, SHAPEFILE_FOLDER)))
shapefile_list = [i for i in shapefile_list if '.shp' in i]
print(shapefile_list)

gdf_list = []
for shapefile_filename in shapefile_list:
  gdf = gpd.read_file(op.join(DATABASE_PATH, DATABASE_NAME, SHAPEFILE_FOLDER, shapefile_filename))
  gdf_list.append(gdf)

global_gdf = gpd.GeoDataFrame( pd.concat( gdf_list, ignore_index = True ) )

# Renaming & selecting the the columns
global_gdf = global_gdf.rename(columns = GDF_MAPPING_DICT)

# Dropping NA and NULL
sorted_gdf = global_gdf.dropna(how='any', subset=list(GDF_MAPPING_DICT.values())).sort_values(by = 'fire_area', ascending = False)
sorted_gdf = sorted_gdf[sorted_gdf['fire_cont_date']!='NULL']

# Extracting the alert year
sorted_gdf['fire_alert_year'] = sorted_gdf['fire_alert_date'].apply(lambda x: x.split('-')[0]).astype('int')

# Filtering the fire since 2017-01-01
sorted_gdf = sorted_gdf[sorted_gdf['fire_alert_year'] > 2015]

# Filtering on big fires (minimum area of one patch 256x256x20m)
sorted_gdf = sorted_gdf[sorted_gdf['fire_area'] > 2600]

# Filter the biggest fires
filtered_gdf = sorted_gdf.head(NUMBER_OF_FIRES).reset_index()

# Check the final gdf
filtered_gdf

['PFTOTCAL_HORSNULL32610.shp']


,index,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,fire_alert_date,fire_cont_date,Shape_Leng,Shape_Area,fire_id,fire_area,BBOX,fire_geometry,fire_alert_year
0,290,2020,CA,USF,MNF,AUGUST COMPLEX FIRES,00008864,2020-08-16,2020-11-11,1.298407e+06,4.179187e+09,291,417918.70,"Polygon ((455998.47609156 4358097.87038731, 53...","MULTIPOLYGON (((525060.419 4358103.964, 524934...",2020
1,8169,2018,CA,CDF,LNU,RANCH,00008646,2018-07-27,2018-09-19,3.482277e+05,1.660030e+09,8170,166003.05,"Polygon ((490516.59312277 4321802.50334735, 54...","MULTIPOLYGON (((525040.645 4321941.501, 525026...",2018
2,175,2020,CA,CDF,SCU,SCU COMPLEX,00005740,2020-08-16,2020-09-11,5.033547e+05,1.605892e+09,176,160589.16,"Polygon ((594434.49001191 4107437.00079614, 66...","MULTIPOLYGON (((600314.004 4152845.714, 600355...",2020
3,247,2020,CA,USF,SNF,CREEK,00001391,2020-09-04,2020-12-24,8.180878e+05,1.537168e+09,248,153716.77,"Polygon ((811330.89762775 4100254.70937166, 85...","MULTIPOLYGON (((839412.588 4112371.716, 839408...",2020
4,242,2020,CA,USF,PNF,NORTH COMPLEX,00001302,2020-08-17,2020-12-03,3.958365e+05,1.290127e+09,243,129012.67,"Polygon ((629896.46019884 4375255.46951663, 69...","MULTIPOLYGON (((652102.740 4377766.501, 652098...",2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,300,2020,CA,NPS,YNP,BLUEJAY,00000054,2020-07-25,2020-11-19,4.408865e+04,2.801240e+07,301,2801.24,"Polygon ((794240.66351095 4189948.41067016, 80...","MULTIPOLYGON (((798652.813 4190517.960, 798657...",2020
105,322,2020,CA,BLM,CCD,NORTH,00030547,2020-08-02,2020-08-10,4.284393e+04,2.785165e+07,323,2785.17,"Polygon ((753405.85427991 4412372.39775962, 76...","POLYGON ((756152.078 4419006.206, 756181.541 4...",2020
106,7106,2016,CA,CDF,FKU,MINERAL,011358,2016-08-08,2016-08-19,3.292598e+04,2.747934e+07,7107,2747.93,"Polygon ((720157.67466306 3998231.60527129, 72...","MULTIPOLYGON (((723898.039 4006044.153, 723867...",2016
107,41,2020,CA,CDF,ORC,BOND,20136890,2020-12-03,2020-12-07,4.384591e+04,2.703745e+07,42,2703.75,"Polygon ((987747.83027922 3742480.12586713, 10...","POLYGON ((996930.765 3746766.923, 996993.165 3...",2020


# Utility functions

In [ ]:
# Fire record extraction
def fire_features_extraction(bdd_index, fire_bdd):
  """
  This function extracts key fire characteristics from a Geo Dataframe.
  INPUTS:
     - bdd_index: index of a record
     - fire_bdd: geo detaframe
  OUTPUT:
     - A dictionary containing the fire characteristics
  """

    fire_name = DATABASE_NAME + '_Sentinel2_' + str(fire_bdd['fire_id'][bdd_index])

    fire_perimeter = fire_bdd['fire_geometry'][bdd_index].bounds
    longitude1, latitude1  = fire_perimeter[0], fire_perimeter[1]
    longitude2, latitude2  = fire_perimeter[2], fire_perimeter[3]
      
    inProj = Proj(init=INDIVIDUAL_SHAPEFILE_CRS_CONFIG)
    outProj = Proj(init='epsg:4326')

    longitude1, latitude1  = transform(inProj , outProj, fire_perimeter[0], fire_perimeter[1])
    longitude2, latitude2  = transform(inProj , outProj, fire_perimeter[2], fire_perimeter[3])

    gps_bounding_box = [longitude1, latitude1, longitude2, latitude2]
    
    fire_cont_date = fire_bdd['fire_cont_date'][bdd_index].replace('/', '-').split(' ')[0]
    fire_alert_date = fire_bdd['fire_alert_date'][bdd_index].replace('/', '-').split(' ')[0]

    fire_cont_date_obj = datetime.strptime(fire_cont_date, '%Y-%m-%d')
    fire_cont_date_a_few_days_after =  fire_cont_date_obj + timedelta(days=30)
    fire_cont_date_a_few_days_after = fire_cont_date_a_few_days_after.strftime('%Y-%m-%d')

    fire_alert_date_obj = datetime.strptime(fire_alert_date, '%Y-%m-%d')
    fire_alert_date_a_few_days_before =  fire_alert_date_obj + timedelta(days=-30)
    fire_alert_date_a_few_days_before = fire_alert_date_a_few_days_before.strftime('%Y-%m-%d')

    return {'fire_name' : fire_name,
            'fire_area' : fire_bdd['fire_area'][bdd_index],
            'fire_perimeter' : gps_bounding_box, 
            'fire_cont_date' : fire_cont_date,
            'fire_cont_date_a_few_days_after' : fire_cont_date_a_few_days_after,
            'fire_alert_date' : fire_alert_date,
            'fire_alert_date_a_few_days_before' : fire_alert_date_a_few_days_before}

In [ ]:
def extract_image(fire_record, bands, before_event):
  """
  This function extracts a Google Earth Engine image.
  INPUTS:
     - fire_record: dictionary of fire record
     - bands: a list containing the channels of interest
     - before_event: boolean choosing the image before and after the fire event
  OUTPUT:
     - image object
     - a bounding box
  """
  
  # Construct the bounding box
  bounding_box = ee.Geometry.Rectangle(fire_record['fire_perimeter'])

  # Defining the date range based on before_event boolean
  if before_event:
    fire_date1 = fire_record['fire_alert_date_a_few_days_before']
    fire_date2 = fire_record['fire_alert_date']
  else:
    fire_date1 = fire_record['fire_cont_date']
    fire_date2 = fire_record['fire_cont_date_a_few_days_after']

  # Collecting the Sentinel image
  image = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterBounds(bounding_box)
          .filterDate(fire_date1, fire_date2)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
          .select(bands)
          .sort('system:index', opt_ascending=False).mosaic()
          .clip(bounding_box))

  return image, bounding_box

In [ ]:
def images_package_extraction(i, fire_bdd):

  """
  This function extracts a package of pre and post fire images
  """

  # Extract a fire record
  fire_record = fire_features_extraction(i, fire_bdd)

  # Collect the image RGB + IR 
  fire_image_RGBIR_postFire, bounding_box  = extract_image(fire_record, ['B2', 'B3', 'B4', 'B7', 'B12'], False)
  fire_image_RGBIR_preFire, bounding_box = extract_image(fire_record, ['B2', 'B3', 'B4', 'B7', 'B12'], True)
  
  return fire_image_RGBIR_preFire, fire_image_RGBIR_postFire, bounding_box, fire_record

In [ ]:
def image_export(fire_image_RGBIR, fire_record, before_event):
  """
  This function exports a Sentinel 2 image to a tif file stored into a drive account.
  """

  if before_event :
    fire_output_filename = fire_record['fire_name'] + '_preFire_' + 'RGBIR'
  else:
    fire_output_filename = fire_record['fire_name'] + '_postFire_' + 'RGBIR'
  
  task_config = {
      'folder': 'GEE_exports',
      'scale': 20,
      'crs': INDIVIDUAL_SHAPEFILE_CRS_CONFIG,
      'maxPixels': 200000000,
      'fileNamePrefix': fire_output_filename,
      'description': 'export_task'
  }

  task = ee.batch.Export.image.toDrive(image = fire_image_RGBIR,
                                       **task_config)
  
  print('Generating the Sentinel tif: ' + fire_output_filename)
  task.start()

  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    print(task.status())
    time.sleep(30)

  return

In [ ]:
def add_ee_layer(self, ee_object, vis_params, name):
  """
  This function defines a method for displaying Earth Engine image tiles on a folium map.
  """

    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
def show_image(vis_params, image, bounding_box):

  """
  This function displays an image on a map.
  """

  # Centering the map on the centroid of the feature map
  map_s2 = folium.Map(location = bounding_box.centroid().getInfo()['coordinates'][::-1], zoom_start = 10)
  
  # Add the image layer to the map and display it.
  map_s2.add_ee_layer( image, vis_params, 'layer')
  
  # Add the fire perimeter
  folium.GeoJson( data = bounding_box.getInfo(), name = 'bounding box', overlay = True, control = True, 
                  style_function = lambda x: {'color': 'red', 'fillColor':'#00000000'} ).add_to(map_s2)

  return map_s2

# Visual check

In [ ]:
vis_params_RGB = {
  'min': 0,
  'max': 2000,
  'bands': ['B2', 'B3', 'B4'][::-1] }

for i in range(0, 10):
  
  # Extracting the images
  fire_image_RGBIR_preFire, fire_image_RGBIR_postFire, bounding_box, fire_record = images_package_extraction(i, filtered_gdf)
  pp.pprint(fire_record)

  # Showing the images
  map_features_preFire = show_image(vis_params_RGB, fire_image_RGBIR_preFire, bounding_box)
  map_features_postFire = show_image(vis_params_RGB, fire_image_RGBIR_postFire, bounding_box)

  from IPython.core.display import display, HTML

  htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 30%; margin: 0 auto; border: 2px solid black"></iframe>'
            '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 30%; margin: 0 auto; border: 2px solid black"></iframe>'
            .format(map_features_preFire.get_root().render().replace('"', '&quot;'),400,400,
                    map_features_postFire.get_root().render().replace('"', '&quot;'),400,400))
  display(htmlmap)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/pytho

{   'fire_alert_date': '2020-08-16',
    'fire_alert_date_a_few_days_before': '2020-07-17',
    'fire_area': 417918.7,
    'fire_cont_date': '2020-11-11',
    'fire_cont_date_a_few_days_after': '2020-12-11',
    'fire_name': 'CAL_database_Sentinel2_291',
    'fire_perimeter': [   -123.51082784298728,
                          39.371227756473516,
                          -122.53795635356124,
                          40.441537128938315]}


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/pytho

{   'fire_alert_date': '2018-07-27',
    'fire_alert_date_a_few_days_before': '2018-06-27',
    'fire_area': 166003.05,
    'fire_cont_date': '2018-09-19',
    'fire_cont_date_a_few_days_after': '2018-10-19',
    'fire_name': 'CAL_database_Sentinel2_8170',
    'fire_perimeter': [   -123.1095883962178,
                          39.045238703196624,
                          -122.43667364733632,
                          39.528976753305706]}


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/pytho

...

{   'fire_alert_date': '2018-07-23',
    'fire_alert_date_a_few_days_before': '2018-06-23',
    'fire_area': 92936.62,
    'fire_cont_date': '2018-09-01',
    'fire_cont_date_a_few_days_after': '2018-10-01',
    'fire_name': 'CAL_database_Sentinel2_7992',
    'fire_perimeter': [   -122.78466999596687,
                          40.493150200327776,
                          -122.39251488996382,
                          40.93670239595063]}


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/pytho

{   'fire_alert_date': '2020-08-19',
    'fire_alert_date_a_few_days_before': '2020-07-20',
    'fire_area': 69058.77,
    'fire_cont_date': '2021-01-04',
    'fire_cont_date_a_few_days_after': '2021-02-03',
    'fire_name': 'CAL_database_Sentinel2_244',
    'fire_perimeter': [   -118.86112682107117,
                          36.09210291506932,
                          -118.28569630471146,
                          36.37464327131643]}


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/pytho

{   'fire_alert_date': '2020-09-08',
    'fire_alert_date_a_few_days_before': '2020-08-09',
    'fire_area': 63709.61,
    'fire_cont_date': '2020-12-12',
    'fire_cont_date_a_few_days_after': '2021-01-11',
    'fire_name': 'CAL_database_Sentinel2_263',
    'fire_perimeter': [   -123.77871995074976,
                          41.740821138921035,
                          -123.29792710765294,
                          42.115150789497655]}


# Sentinel tiff generation

In [ ]:
# Choosing the fire indices to extract
start_index = 0
#finish_index = NUMBER_OF_FIRES
finish_index = 108

# Looping on every fire in the geodataframe
for i in tqdm(range(start_index, finish_index)):

  # Extractig the images from Google Earth Engine
  fire_image_RGBIR_preFire, fire_image_RGBIR_postFire, bounding_box, fire_record = images_package_extraction(i, filtered_gdf)
  pp.pprint(fire_record)
  print(str(i) + ' / ' + str(filtered_gdf.shape[0] - 1))
  
  # Exporting the images in tif files
  #image_export(fire_image_RGBIR_preFire, fire_record, True)
  image_export(fire_image_RGBIR_postFire, fire_record, False)

  0%|          | 0/108 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pypr

{   'fire_alert_date': '2020-08-16',
    'fire_alert_date_a_few_days_before': '2020-07-17',
    'fire_area': 417918.7,
    'fire_cont_date': '2020-11-11',
    'fire_cont_date_a_few_days_after': '2020-12-11',
    'fire_name': 'CAL_database_Sentinel2_291',
    'fire_perimeter': [   -123.51082784298728,
                          39.371227756473516,
                          -122.53795635356124,
                          40.441537128938315]}
0 / 108
Generating the Sentinel tif: CAL_database_Sentinel2_291_postFire_RGBIR
Polling for task (id: 2SGRHFFGVBX5PBQCLD4T43FS).
{'state': 'READY', 'description': 'export_task', 'creation_timestamp_ms': 1642071091203, 'update_timestamp_ms': 1642071091203, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '2SGRHFFGVBX5PBQCLD4T43FS', 'name': 'projects/earthengine-legacy/operations/2SGRHFFGVBX5PBQCLD4T43FS'}
Polling for task (id: 2SGRHFFGVBX5PBQCLD4T43FS).
{'state': 'RUNNING', 'description': 'export_task', 'creation_timestamp_ms': 1642071091203, 

  1%|          | 1/108 [06:04<10:50:16, 364.64s/it]/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgra

{   'fire_alert_date': '2018-07-27',
    'fire_alert_date_a_few_days_before': '2018-06-27',
    'fire_area': 166003.05,
    'fire_cont_date': '2018-09-19',
    'fire_cont_date_a_few_days_after': '2018-10-19',
    'fire_name': 'CAL_database_Sentinel2_8170',
    'fire_perimeter': [   -123.1095883962178,
                          39.045238703196624,
                          -122.43667364733632,
                          39.528976753305706]}
1 / 108
Generating the Sentinel tif: CAL_database_Sentinel2_8170_postFire_RGBIR
Polling for task (id: IFFKJ5AAKRTXI7QOG7BR4RBJ).
{'state': 'READY', 'description': 'export_task', 'creation_timestamp_ms': 1642071455881, 'update_timestamp_ms': 1642071455881, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'IFFKJ5AAKRTXI7QOG7BR4RBJ', 'name': 'projects/earthengine-legacy/operations/IFFKJ5AAKRTXI7QOG7BR4RBJ'}
Polling for task (id: IFFKJ5AAKRTXI7QOG7BR4RBJ).
{'state': 'READY', 'description': 'export_task', 'creation_timestamp_ms': 1642071455881, 

  2%|▏         | 2/108 [07:06<5:29:10, 186.33s/it] /usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgra

...

 98%|█████████▊| 106/108 [2:37:27<02:39, 79.54s/it]/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgra

{   'fire_alert_date': '2016-08-08',
    'fire_alert_date_a_few_days_before': '2016-07-09',
    'fire_area': 2747.93,
    'fire_cont_date': '2016-08-19',
    'fire_cont_date_a_few_days_after': '2016-09-18',
    'fire_name': 'CAL_database_Sentinel2_7107',
    'fire_perimeter': [   -120.55428699105988,
                          36.10380334044085,
                          -120.49154896592728,
                          36.18365446158778]}
106 / 108
Generating the Sentinel tif: CAL_database_Sentinel2_7107_postFire_RGBIR
Polling for task (id: GW7UJCJUUUWCDYHLZICDGZAH).
{'state': 'READY', 'description': 'export_task', 'creation_timestamp_ms': 1642080538769, 'update_timestamp_ms': 1642080538769, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GW7UJCJUUUWCDYHLZICDGZAH', 'name': 'projects/earthengine-legacy/operations/GW7UJCJUUUWCDYHLZICDGZAH'}


 99%|█████████▉| 107/108 [2:37:58<01:04, 64.98s/it]/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgra

{   'fire_alert_date': '2020-12-03',
    'fire_alert_date_a_few_days_before': '2020-11-03',
    'fire_area': 2703.75,
    'fire_cont_date': '2020-12-07',
    'fire_cont_date_a_few_days_after': '2021-01-06',
    'fire_name': 'CAL_database_Sentinel2_42',
    'fire_perimeter': [   -117.7390241388629,
                          33.710262329450316,
                          -117.6024738223089,
                          33.768104566044855]}
107 / 108
Generating the Sentinel tif: CAL_database_Sentinel2_42_postFire_RGBIR
Polling for task (id: AIND6YHKHZ7Q2AURX3EBN52P).
{'state': 'READY', 'description': 'export_task', 'creation_timestamp_ms': 1642080569756, 'update_timestamp_ms': 1642080569756, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'AIND6YHKHZ7Q2AURX3EBN52P', 'name': 'projects/earthengine-legacy/operations/AIND6YHKHZ7Q2AURX3EBN52P'}
Polling for task (id: AIND6YHKHZ7Q2AURX3EBN52P).
{'state': 'READY', 'description': 'export_task', 'creation_timestamp_ms': 1642080569756, 'upda

100%|██████████| 108/108 [2:38:59<00:00, 88.33s/it]


# Label tiff generation

Check the input data

In [ ]:
IMAGES_FOLDER = '/content/gdrive/MyDrive/Data/Training_data/CAL_database_2/images'
MASKS_FOLDER = '/content/gdrive/MyDrive/Data/Training_data/CAL_database_2/masks'
INDIVIDUAL_SHAPEFILE_FOLDER = '/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/CAL_database/individual_shapefiles'

# Listing all Sentinel tif files
sentinel_tif_list = os.listdir(IMAGES_FOLDER)
sentinel_tif_list = [i for i in sentinel_tif_list if '.tif' in i]

# Listing all Shapefiles
shapefile_list = os.listdir(INDIVIDUAL_SHAPEFILE_FOLDER)
shapefile_list = [i for i in shapefile_list if '.shp' in i]

print('Total number of Sentinel images : ', len(sentinel_tif_list))
print('Total number of Shapefiles : ', len(shapefile_list))

Total number of Sentinel images :  41
Total number of Shapefiles :  600


Generation of the label tiff

In [ ]:
# Init driver
drv_tiff = gdal.GetDriverByName("GTiff")
driver = ogr.GetDriverByName('ESRI Shapefile')

for sentinel_tif_file in tqdm(sentinel_tif_list):

  print(sentinel_tif_file)

  # Opening the Sentinel image
  fire_sentinel_raster = gdal.Open(op.join(IMAGES_FOLDER, sentinel_tif_file))
  geot = fire_sentinel_raster.GetGeoTransform()

  # Finding and opening the Shapefile
  fire_id = sentinel_tif_file.split('_')[3]
  shapefile_filename = 'fperim' + str(fire_id) + '.shp'
  shapefile_filename = op.join(INDIVIDUAL_SHAPEFILE_FOLDER, shapefile_filename)
  print(shapefile_filename)

  dataSource = driver.Open(shapefile_filename, 0) # 0 means read-only. 1 means writeable.

  # Checking if the corresponding shapefile has been found.
  shapefile_not_found = []
  if dataSource is None:
    shapefile_not_found.append(fire_id)
    break
  else:
    layer = dataSource.GetLayer()

  # Defining the label filename
  if 'post' in sentinel_tif_file:
    label_filename = op.join(MASKS_FOLDER, 'CAL_database_Sentinel2_' + fire_id + '_postFire_label.tif')
  else:
    label_filename = op.join(MASKS_FOLDER, 'CAL_database_Sentinel2_' + fire_id + '_preFire_label.tif')

  # Creating the new raster
  chn_ras_ds = drv_tiff.Create(label_filename, 
                               fire_sentinel_raster.RasterXSize, 
                               fire_sentinel_raster.RasterYSize, 
                               1, 
                               gdal.GDT_Byte)
  
  chn_ras_ds.SetGeoTransform(geot)
  chn_ras_ds.SetProjection(fire_sentinel_raster.GetProjection())

  if 'post' in sentinel_tif_file: gdal.RasterizeLayer(chn_ras_ds, [1], layer, None)
  chn_ras_ds.GetRasterBand(1).SetNoDataValue(0.0)

  chn_ras_ds = None

# Checking the number of files generated
label_tif_list = os.listdir(MASKS_FOLDER)
print('Current generated Label tif files: ' + str(len(label_tif_list)))
print('Total submitted Sentinel tif files: ' + str(len(sentinel_tif_list)))
print('Number of unfound shapefile: ' + str(len(shapefile_not_found)))

# Finally, visual check the postFire label tif because some are totally black for an unknown reason.

  0%|          | 0/41 [00:00<?, ?it/s]

CAL_database_Sentinel2_176_postFire_RGBIR.tif
/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/CAL_database/individual_shapefiles/fperim176.shp


  2%|▏         | 1/41 [00:00<00:19,  2.02it/s]

CAL_database_Sentinel2_155_postFire_RGBIR.tif
/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/CAL_database/individual_shapefiles/fperim155.shp


  5%|▍         | 2/41 [00:01<00:40,  1.04s/it]

...

/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/CAL_database/individual_shapefiles/fperim8326.shp
CAL_database_Sentinel2_8578_postFire_RGBIR.tif


 98%|█████████▊| 40/41 [02:08<00:00,  2.62it/s]

/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/CAL_database/individual_shapefiles/fperim8578.shp
CAL_database_Sentinel2_8351_postFire_RGBIR.tif


100%|██████████| 41/41 [02:08<00:00,  3.14s/it]

/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/CAL_database/individual_shapefiles/fperim8351.shp
Current generated Label tif files: 41
Total submitted Sentinel tif files: 41
Number of unfound shapefile: 0
